In [ ]:
import pandas as pd
import numpy as np


In [3]:
# Census Data missing some places/states that did not have census info
# census_full.csv includes those missing, but decided not to support those in model
census_data = pd.read_csv("./Data/census.csv", dtype={
	"City": str,
	"State": str,
	"PlaceCode": str,
	"StateCode": str,
	"DP1_0025C": float,
	"DP1_0049C": float,
	"DP1_0001C": float,
	"DP1_0006P": float,
	"DP1_0007P": float,
	"DP1_0008P": float,
	"DP1_0009P": float,
	"DP1_0010P": float,
	"DP1_0011P": float,
	"DP1_0012P": float,
	"DP1_0013P": float,
	"DP1_0014P": float,
	"DP1_0015P": float,
	"DP1_0016P": float,
	"DP1_0017P": float,
	"DP1_0018P": float,
	"DP1_0019P": float,
	"DP1_0022P": float,
	"DP1_0078P": float,
	"DP1_0079P": float,
	"DP1_0081P": float,
	"DP1_0080P": float,
	"DP1_0082P": float,
	"DP1_0114P": float,
	"DP1_0120P": float,
	"DP1_0133P": float,
	"DP1_0134P": float,
	"DP1_0117P": float,
	"DP1_0118P": float,
	"DP1_0093P": float,
})

In [15]:
yelp_data = pd.read_json("Data/Yelp JSON/yelp_academic_dataset_business.json", lines=True) 
# Chose to ignore hours and attributes in mode, harder to get predicitve features
yelp_data.drop(["business_id", "address", "postal_code", "latitude", "longitude", "is_open", "attributes", "hours"],
               axis=1, inplace=True)

In [13]:
yelp_data.columns

Index(['city', 'state', 'stars', 'review_count', 'categories'], dtype='object')

In [17]:
IV_data = yelp_data[yelp_data["city"] == "Isla Vista"]

In [20]:
for row in IV_data.itertuples():
    print(f"{row.name}: {row.categories}")

California Tacos and Taproom: Mexican, Beer Bar, Bars, Sports Bars, Nightlife, Restaurants, Tacos
Isla Vista Community Bike Center: Community Service/Non-Profit, Shopping, Local Services, Bike Rentals, Bikes, Bike Repair/Maintenance, Active Life, Sporting Goods
Pho Bistro: Food, Restaurants, Chinese, Bubble Tea, Vietnamese
Su’s Bowl: Chinese, Restaurants, Asian Fusion, Szechuan
Wells Fargo Bank: Banks & Credit Unions, Financial Services
Red Pepper Chinese Food Express: Restaurants, Fast Food, Chinese
The Loop- Coastal Ridge: Real Estate Services, Apartments, Real Estate, Home Services
Santorini Greek Island Grill: Mediterranean, Food, Restaurants, Greek, Specialty Food, Ethnic Food
Subway: Restaurants, Sandwiches, Fast Food
The Study Hall: Bars, Nightlife
Wanderlust Hair Studio/Gallery: Blow Dry/Out Services, Men's Hair Salons, Beauty & Spas, Hair Salons, Hair Stylists
Cajé Coffee Roasters - Isla Vista: Food, Internet Cafes, Coffee & Tea, Acai Bowls, Juice Bars & Smoothies, Bagels
Kapt

In [21]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

c:\Users\adamw\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\adamw\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adamw\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to ru

[[ 0.02900643 -0.03969981  0.02920654 ...  0.04757527 -0.01917132
  -0.01472931]
 [ 0.01358889 -0.07393813  0.0298319  ...  0.01244734  0.07311949
  -0.01819833]
 [ 0.08341126 -0.11538616  0.00016828 ... -0.02274583  0.00490523
   0.05907445]]


In [24]:
def cosSim(A, B):
    return np.dot(A, B)/(np.linalg.norm(A) * np.linalg.norm(B))

In [ ]:
businesses = ['Caje', 'Blenders', 'George']
categories = ['Food, Internet Cafes, Coffee & Tea, Acai Bowls, Juice Bars & Smoothies, Bagels',
 'Juice Bars & Smoothies, American (Traditional), Food, Restaurants',
 'Real Estate, Property Management, University Housing, Apartments, Real Estate Agents, Home Services']
embeddings = model.encode(categories)

In [30]:
print(f"Caje/Blender: {cosSim(embeddings[0], embeddings[1])}")
print(f"Caje/George: {cosSim(embeddings[0], embeddings[2])}")

Caje/Blender: 0.7490094900131226
Caje/George: 0.11996615678071976


In [32]:
embeddings = model.encode(yelp_data['categories'].astype(str).tolist(), show_progress_bar=True)

# Convert embeddings to a DataFrame with 384 columns
embedding_df = pd.DataFrame(embeddings, columns=[f'cat_emb_{i}' for i in range(384)])

Batches: 100%|██████████| 4699/4699 [04:20<00:00, 18.06it/s]


In [ ]:
embedding_df.to_csv("./Data/embeddings.csv", index=False)